Jupyter Workspaces can have complete sections where you are explaining what you are doing in each cell. You can break up your code into executable steps, which some people find helps them with debugging. 

In [ ]:
import requests
import pandas as pd
import domojupyter as domo

# 1. Configuration
# It is best practice to store these in a safe place or a protected cell
API_KEYS = ["YOUR_FIRST_KEY", "YOUR_SECOND_KEY"] # List out as many credentials as there are accounts
ENDPOINT_URL = "https://sports.highlightly.net/football/matches" #Can be a single endpoint or multiple endpoints
DATASET_NAME = "Aggregated Highlightly Matches"

def fetch_highlightly_data(api_key):
    """Fetches data from Highlightly for a specific API key."""
    headers = {
        "x-rapidapi-key": api_key,
        "Accept": "application/json"
    }
    
    # Example params: Adjust based on your specific needs (season, league, etc.)
    params = {"limit": 100, "season": 2025}
    
    try:
        response = requests.get(ENDPOINT_URL, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        
        # Flattening the JSON into a DataFrame
        # Adjust 'data' to the specific key in the response that holds the list
        return pd.DataFrame(data.get('data', []))
    
    except Exception as e:
        print(f"Error fetching data with key {api_key[:5]}... : {e}")
        return pd.DataFrame()

# 2. Main execution logic
all_dfs = []

for key in API_KEYS:
    print(f"Fetching data for account: {key[:5]}...")
    df = fetch_highlightly_data(key)
    
    if not df.empty:
        # Optional: Add a column to identify which account the data came from
        df['source_account_key_prefix'] = key[:5]
        all_dfs.append(df)

# 3. Concatenate and Write to Domo
if all_dfs:
    final_df = pd.concat(all_dfs, ignore_index=True)
    
    print(f"Total rows to write: {len(final_df)}")
    
    # Using the domojupyter library to write the dataset
    # This creates or updates a dataset directly in your Domo instance
    domo.write_dataframe(final_df, 'Aggregated Highlightly Matches')
    print("Successfully wrote aggregated data to Domo.")
else:
    print("No data collected. Skipping Domo write.")